# Collect Data
Extract story trees from Wikipedia articles

In [1]:
# sample Wikipedia articles about famous biographies
URLs = ["https://en.wikipedia.org/wiki/Pieter_Bruegel_the_Elder", "https://en.wikipedia.org/wiki/Vincent_van_Gogh",
        "https://en.wikipedia.org/wiki/Rembrandt", "https://en.wikipedia.org/wiki/Émile_Zola"]
# pick one of articles as a seed entity to be the focus of the generated story
seed_url = URLs[0]
print (seed_url)

https://en.wikipedia.org/wiki/Pieter_Bruegel_the_Elder


In [2]:
# get WikiData ID
import requests, json
wiki_api_query = "https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&format=json&titles=%s"

def link_wiki_data(url):
    label = url.strip('/').split('/')[-1]
    response = requests.get(wiki_api_query%label)
    URI = json.loads(response.text)['entities'].keys()[0]
    return label, URI

seed_label, seed_URI = link_wiki_data(seed_url)
print seed_label, seed_URI

Pieter_Bruegel_the_Elder Q43270


In [4]:
# get the corresponding Wiki page
page = requests.get(seed_url).text

# scrape the links
import scrape
# collect story trees with the corresponding WikiData IDs
def get_wiki_tree(scraper):
    '''
    scraper is a function extracting links from a Wiki page
    '''
    s_links = scraper(page)
#     print(p_links)
    story_tree = {}
    for k, p_links in enumerate(s_links):
        for i, paragraph in enumerate(p_links):
            for j, sentence in enumerate(paragraph):
                URIs = []
                for link in sentence:
                    # WikiData ID
        #             _, URI = link_wiki_data(link['url'])
                    # Wikipedia article title
#                     print(link)
                    URI = link['title']
                    # skip entities missing from WikiData
                    if URI != '-1':
                        URIs.append(URI)
        #                 print(URI)
                if len(URIs) > 0:
                    if k not in story_tree:
                        story_tree[k] = {}
                    if i not in story_tree[k]:
                        story_tree[k][i] = {}
                    story_tree[k][i][j] = URIs
    return story_tree

print("Extracting story trees")

# get only the links from the article summary
# summary_tree = get_wiki_sequence(scrape.extract_summary_paragraphs_links)
# print("summary tree with %d branches extracted"%len(summary_tree))
# print(summary_tree)

# get only the links from the article without the summary
story_tree= get_wiki_tree(scrape.extract_story_paragraphs_links)
print("story tree with %d branches extracted"%len(story_tree))
import pprint
pprint.pprint(story_tree)

Extracting story trees
story tree with 8 branches extracted
{2: {0: {0: [u'Lodovico Guicciardini',
             u'Karel van Mander',
             u'Schilder-boeck'],
         1: [u'Breda']},
     1: {1: [u'House of Orange-Nassau'], 2: [u'Renaissance humanism']},
     2: {1: [u'Pieter Coecke van Aelst', u'Mayken Coecke']},
     3: {1: [u'Mechelen', u'Peeter Baltens', u'Grisaille'],
         2: [u'Mayken Verhulst']},
     4: {0: [u'Guild of Saint Luke'],
         2: [u'Reggio Calabria'],
         3: [u'Sicily'],
         4: [u'Giulio Clovio']},
     5: {0: [u'Hieronymus Cock']},
     6: {2: [u'Renaissance humanism', u'Blackletter']},
     7: {4: [u'Antoine Perrenot de Granvelle', u'Habsburg'],
         6: [u'Pieter Brueghel the Younger', u'Jan Brueghel the Elder'],
         7: [u'Kapellekerk']}},
 3: {0: {1: [u'Renaissance humanism'],
         2: [u'Michelangelo', u'Leonardo da Vinci'],
         3: [u'Martin Luther',
             u'Ninety-five Theses',
             u'Protestant Reformati